# `USGS API Example with Parsing`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

In [145]:
import requests
# from concurrent.futures import ThreadPoolExecutor, as_completed
from time import time
import pandas as pd
import json
import time
import datetime 
import re

In [146]:
usgs_url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson'

# `Regular Scrape`

# ` ¯\_(ツ)_/¯`

In [147]:
gg=requests.get(usgs_url)


In [ ]:
# gg.json()

# gg.json().keys()

In [206]:

data_updated=pd.DataFrame(gg.json()['features']).drop(['type'],axis=1)


flatten_dict_df=pd.json_normalize(json.loads(data_updated.to_json(orient="records")))

flatten_dict_df.head(10)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,properties.cdi,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,ak023aj1qdtp,2.00,Alaska Peninsula,1692302352938,1692302424445,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,",origin,phase-data,",NaN,NaN,0.4000,NaN,ml,earthquake,M 2.0 - Alaska Peninsula,Point,"[-155.1113, 58.2723, 1.1]"
1,ci39643162,3.25,"11 km WSW of Grapevine, CA",1692302340010,1692303557829,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,21.0,3.7,...,",dyfi,focal-mechanism,nearby-cities,origin,pha...",98.0,0.1217,0.3300,20.00,ml,earthquake,"M 3.3 - 11 km WSW of Grapevine, CA",Point,"[-119.0384979, 34.8969994, 9.06]"
2,nn00864607,1.00,"32 km SE of Mina, Nevada",1692302297021,1692302453657,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,",origin,phase-data,",8.0,0.0850,0.0864,171.16,ml,earthquake,"M 1.0 - 32 km SE of Mina, Nevada",Point,"[-117.8963, 38.15, 6.1]"
3,ak023aj1iz4s,1.90,Alaska Peninsula,1692300296609,1692300393554,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,",origin,phase-data,",NaN,NaN,0.3400,NaN,ml,earthquake,M 1.9 - Alaska Peninsula,Point,"[-154.9662, 58.3052, 3.7]"
4,nc73925211,1.86,"2 km SSW of Tres Pinos, CA",1692298696770,1692303136863,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,",nearby-cities,origin,phase-data,scitech-link,",25.0,NaN,0.0900,118.00,md,earthquake,"M 1.9 - 2 km SSW of Tres Pinos, CA",Point,"[-121.3314972, 36.7739983, 5.61]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10588,us7000kgn3,4.00,"4 km SSW of Peştişani, Romania",1689712255158,1691599802187,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,28.0,5.8,...,",dyfi,origin,phase-data,",47.0,1.1380,1.0400,36.00,mb,earthquake,"M 4.0 - 4 km SSW of Peştişani, Romania",Point,"[23.0206, 45.0307, 12.943]"
10589,av91110598,1.08,"72 km WSW of Nikolski, Alaska",1689712024650,1689721340550,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,",origin,phase-data,",5.0,NaN,0.3000,160.00,ml,earthquake,"M 1.1 - 72 km WSW of Nikolski, Alaska",Point,"[-169.9148333333, 52.815, 4.53]"
10590,ak02395g07mh,1.10,"22 km ENE of Chase, Alaska",1689711958024,1690347720428,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,",origin,phase-data,",NaN,NaN,0.7400,NaN,ml,earthquake,"M 1.1 - 22 km ENE of Chase, Alaska",Point,"[-149.7022, 62.5292, 7.5]"
10591,us6000kvpj,4.10,"99 km WNW of Hihifo, Tonga",1689711830230,1691599489040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,",origin,phase-data,",35.0,3.1490,0.5700,53.00,mb,earthquake,"M 4.1 - 99 km WNW of Hihifo, Tonga",Point,"[-174.5959, -15.5005, 163.646]"


# `What is this 13 digit time?`

+ This is called `Epoch time` or `Unix Epoch` Time where you have `10 digit` (seconds), `13 digit` (milli-seconds) and even `16 digit` (micro-seconds).

+ Interesting how this came about because it stems from the amount time that passed since January 1, 1970.

In [149]:
# Update times from epoch to human readable:

ff=datetime.datetime.fromtimestamp(1692218940296/1000).strftime('%m-%d-%Y %H:%M:%S')


ff


'08-16-2023 14:49:00'

# `Let us distinguish what we are doing below:`

+ `dt.tz_localize` we are starting with `naive` or `unaware` time agnostic of any timezone and make it `aware` such as `UTC` 
    + You are not changing the time but actually, appending the `+00:00` formatting where the next step will convert to your choice

+ `dt.tz_convert` taking an `aware` time object and creating a`time zone` for it

In [205]:
flatten_dict_df['properties.time']=pd.to_datetime(flatten_dict_df['properties.time'],unit='ms').dt\
.tz_localize('UTC').dt.tz_convert('US/Mountain')

# .dt.tz_convert('Asia/Kuala_Lumpur')

flatten_dict_df['properties.updated']=pd.to_datetime(flatten_dict_df['properties.updated'],unit='ms').dt\
.tz_localize('UTC').dt.tz_convert('US/Mountain')

# .dt.tz_convert('US/Mountain')
# 
# pd.to_datetime(flatten_dict_df['properties.updated'],unit='ms',tz='US/Mountain')

# flatten_dict_df
flatten_dict_df[flatten_dict_df['properties.place'].str.contains('colombia',case=False)]

# https://eclipse.gsfc.nasa.gov/SEhelp/TimeZone.html

,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,properties.cdi,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
13,us7000kp2s,4.8,"12 km NW of Villavicencio, Colombia",2023-08-17 11:40:16.323000-06:00,2023-08-17 14:18:58.223000-06:00,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,2.0,4.3,...,",dyfi,origin,phase-data,",81.0,0.876,1.17,70.0,mb,earthquake,"M 4.8 - 12 km NW of Villavicencio, Colombia",Point,"[-73.6879, 4.2333, 10]"
20,us7000kp2j,5.7,"6 km SW of Restrepo, Colombia",2023-08-17 11:17:18.031000-06:00,2023-08-17 14:21:37.454000-06:00,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,33.0,5.2,...,",dyfi,ground-failure,losspager,origin,phase-da...",79.0,0.949,0.98,77.0,mww,earthquake,"M 5.7 - 6 km SW of Restrepo, Colombia",Point,"[-73.6065, 4.2142, 7.942]"
23,us7000kp2i,6.3,"16 km N of Cumaral, Colombia",2023-08-17 11:04:49.641000-06:00,2023-08-17 14:22:12.128000-06:00,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,196.0,7.1,...,",dyfi,ground-failure,losspager,moment-tensor,o...",131.0,0.912,1.35,39.0,mww,earthquake,"M 6.3 - 16 km N of Cumaral, Colombia",Point,"[-73.5111, 4.4177, 10]"
59,us7000kp07,4.6,northern Colombia,2023-08-17 07:28:36.571000-06:00,2023-08-17 12:30:34.936000-06:00,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,6.0,3.4,...,",dyfi,origin,phase-data,",49.0,1.140,0.82,105.0,mb,earthquake,M 4.6 - northern Colombia,Point,"[-75.5938, 7.3366, 54.138]"
948,us6000kzwn,4.9,"15 km WNW of Valdivia, Colombia",2023-08-14 09:55:10.597000-06:00,2023-08-17 11:43:57.809000-06:00,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,4.0,3.4,...,",dyfi,origin,phase-data,",158.0,1.149,0.80,83.0,mb,earthquake,"M 4.9 - 15 km WNW of Valdivia, Colombia",Point,"[-75.5221, 7.3477, 48.084]"
1288,us6000kzpp,4.3,"136 km WNW of Salahonda, Colombia",2023-08-13 06:58:42.786000-06:00,2023-08-13 07:32:41.040000-06:00,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,",origin,phase-data,",33.0,2.749,0.75,137.0,mb,earthquake,"M 4.3 - 136 km WNW of Salahonda, Colombia",Point,"[-79.6821, 2.7223, 10]"
1767,us6000kzf4,4.9,"19 km WNW of Valdivia, Colombia",2023-08-11 11:59:59.210000-06:00,2023-08-12 09:34:01.197000-06:00,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,2.0,2.5,...,",dyfi,moment-tensor,origin,phase-data,",93.0,1.164,0.72,134.0,mwr,earthquake,"M 4.9 - 19 km WNW of Valdivia, Colombia",Point,"[-75.556, 7.3617, 46.685]"
1985,us6000kz6d,4.3,northern Colombia,2023-08-10 19:27:15.106000-06:00,2023-08-10 20:26:26.040000-06:00,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,",origin,phase-data,",32.0,0.960,0.87,129.0,mb,earthquake,M 4.3 - northern Colombia,Point,"[-73.0839, 6.8585, 145.692]"
3338,us6000ky9h,4.3,"11 km SE of Piedecuesta, Colombia",2023-08-06 05:09:28.786000-06:00,2023-08-06 05:23:54.040000-06:00,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,",origin,phase-data,",47.0,1.011,0.75,112.0,mb,earthquake,"M 4.3 - 11 km SE of Piedecuesta, Colombia",Point,"[-72.9849, 6.9055, 143.798]"
4023,us6000kxr4,4.2,northern Colombia,2023-08-03 23:47:15.686000-06:00,2023-08-16 06:21:34.040000-06:00,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,",origin,phase-data,",46.0,1.019,0.65,126.0,mb,earthquake,M 4.2 - northern Colombia,Point,"[-7

In [29]:
flatten_dict_df['properties.time']=pd.to_datetime(flatten_dict_df['properties.time'],unit='ms')

flatten_dict_df['properties.updated']=pd.to_datetime(flatten_dict_df['properties.updated'],unit='ms')

flatten_dict_df=flatten_dict_df.rename(columns={'properties.time':'Timestamp','properties.updated':'Updated_Time'})

flatten_dict_df

# Need to check the correct time zone information...

KeyError: 'properties.time'

In [30]:
flatten_dict_df['geometry.coordinates']

coords_df=pd.DataFrame(flatten_dict_df['geometry.coordinates'].to_list(),
                                                    columns=['longitude', 'latitude','depth_km'])


updated_df=pd.concat([flatten_dict_df, coords_df], axis=1)

updated_df

,id,properties.mag,properties.place,Timestamp,Updated_Time,properties.tz,properties.url,properties.detail,properties.felt,properties.cdi,...,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates,longitude,latitude,depth_km
0,ak023aj1iz4s,1.90,Alaska Peninsula,2023-08-17 19:24:56.609,2023-08-17 19:26:33.554,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,0.34,NaN,ml,earthquake,M 1.9 - Alaska Peninsula,Point,"[-154.9662, 58.3052, 3.7]",-154.966200,58.305200,3.70
1,nc73925211,1.86,"2 km SSW of Tres Pinos, CA",2023-08-17 18:58:16.770,2023-08-17 19:11:11.520,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,0.09,118.0,md,earthquake,"M 1.9 - 2 km SSW of Tres Pinos, CA",Point,"[-121.3314972, 36.7739983, 5.61]",-121.331497,36.773998,5.61
2,ak023aj145m6,2.50,"8 km WNW of Houston, Alaska",2023-08-17 18:55:42.061,2023-08-17 18:57:15.078,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,0.45,NaN,ml,earthquake,"M 2.5 - 8 km WNW of Houston, Alaska",Point,"[-149.9743, 61.6454, 25.6]",-149.974300,61.645400,25.60
3,ak023aj0vlpp,2.40,"36 km NW of Valdez, Alaska",2023-08-17 18:15:51.837,2023-08-17 18:17:34.650,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,0.93,NaN,ml,earthquake,"M 2.4 - 36 km NW of Valdez, Alaska",Point,"[-146.7538, 61.3902, 15]",-146.753800,61.390200,15.00
4,ak023aj0t643,1.40,"10 km NW of Trapper Creek, Alaska",2023-08-17 18:04:16.512,2023-08-17 18:06:54.312,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,0.54,NaN,ml,earthquake,"M 1.4 - 10 km NW of Trapper Creek, Alaska",Point,"[-150.3462, 62.3913, 63.1]",-150.346200,62.391300,63.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10586,av91110943,0.37,"71 km WSW of Nikolski, Alaska",2023-07-18 19:57:20.320,2023-07-21 02:26:55.280,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,0.23,179.0,ml,earthquake,"M 0.4 - 71 km WSW of Nikolski, Alaska",Point,"[-169.9038333333, 52.8093333333, 5.42]",-169.903833,52.809333,5.42
10587,av91110938,0.08,"Fox Islands, Aleutian Islands, Alaska",2023-07-18 19:56:47.700,2023-07-21 02:19:56.650,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,0.13,333.0,ml,earthquake,"M 0.1 - Fox Islands, Aleutian Islands, Alaska",Point,"[-169.847, 52.787, 7.02]",-169.847000,52.787000,7.02
10588,us6000kvpn,4.60,"145 km SSE of ‘Ohonua, Tonga",2023-07-18 19:51:40.963,2023-08-09 16:18:14.040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,0.54,90.0,mb,earthquake,"M 4.6 - 145 km SSE of ‘Ohonua, Tonga",Point,"[-174.3776, -22.544, 10]",-174.377600,-22.544000,10.00
10589,hv73487487,1.82,"Island of Hawaii, Hawaii",2023-07-18 19:51:13.690,2023-07-18 19:54:14.840,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,NaN,NaN,...,0.10,126.0,md,earthquake,"M 1.8 - Island of Hawaii, Hawaii",Point,"[-155.4691619873, 19.1873340607, 30.2299995422]",-155.469162,19.187334,30.23


In [31]:
# rename columns and clean some of the text with useless commas

updated_df.columns

Index(['id', 'properties.mag', 'properties.place', 'Timestamp', 'Updated_Time',
       'properties.tz', 'properties.url', 'properties.detail',
       'properties.felt', 'properties.cdi', 'properties.mmi',
       'properties.alert', 'properties.status', 'properties.tsunami',
       'properties.sig', 'properties.net', 'properties.code', 'properties.ids',
       'properties.sources', 'properties.types', 'properties.nst',
       'properties.dmin', 'properties.rms', 'properties.gap',
       'properties.magType', 'properties.type', 'properties.title',
       'geometry.type', 'geometry.coordinates', 'longitude', 'latitude',
       'depth_km'],
      dtype='object')

In [32]:
# Find Number of Null values by column
updated_df.isna().sum()

id                          0
properties.mag              0
properties.place            0
Timestamp                   0
Updated_Time                0
properties.tz           10591
properties.url              0
properties.detail           0
properties.felt         10153
properties.cdi          10153
properties.mmi          10471
properties.alert        10535
properties.status           0
properties.tsunami          0
properties.sig              0
properties.net              0
properties.code             0
properties.ids              0
properties.sources          0
properties.types            0
properties.nst           2876
properties.dmin          4956
properties.rms              0
properties.gap           2876
properties.magType          0
properties.type             0
properties.title            0
geometry.type               0
geometry.coordinates        0
longitude                   0
latitude                    0
depth_km                    0
dtype: int64

In [33]:
# Proportion of Null per column
updated_df.isna().sum()/len(updated_df)

id                      0.000000
properties.mag          0.000000
properties.place        0.000000
Timestamp               0.000000
Updated_Time            0.000000
properties.tz           1.000000
properties.url          0.000000
properties.detail       0.000000
properties.felt         0.958644
properties.cdi          0.958644
properties.mmi          0.988670
properties.alert        0.994712
properties.status       0.000000
properties.tsunami      0.000000
properties.sig          0.000000
properties.net          0.000000
properties.code         0.000000
properties.ids          0.000000
properties.sources      0.000000
properties.types        0.000000
properties.nst          0.271551
properties.dmin         0.467944
properties.rms          0.000000
properties.gap          0.271551
properties.magType      0.000000
properties.type         0.000000
properties.title        0.000000
geometry.type           0.000000
geometry.coordinates    0.000000
longitude               0.000000
latitude  

In [34]:
# Drop columns with over 40% null

# decide to drop this column because only 4 rows were Tsunmai vs total rows 0.03% is <1/10 of 1%
len(updated_df[updated_df['properties.tsunami']==1])/len(updated_df)*100

cols_to_drop=['properties.tz','properties.felt','properties.cdi','properties.mmi','properties.alert',
'properties.dmin','geometry.type','properties.type','properties.status',
'properties.code','properties.net','properties.ids','properties.sources',
'properties.tsunami','properties.types','geometry.coordinates']

updated_df=updated_df.drop(cols_to_drop, axis=1)


In [35]:
updated_df

,id,properties.mag,properties.place,Timestamp,Updated_Time,properties.url,properties.detail,properties.sig,properties.nst,properties.rms,properties.gap,properties.magType,properties.title,longitude,latitude,depth_km
0,ak023aj1iz4s,1.90,Alaska Peninsula,2023-08-17 19:24:56.609,2023-08-17 19:26:33.554,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,56,NaN,0.34,NaN,ml,M 1.9 - Alaska Peninsula,-154.966200,58.305200,3.70
1,nc73925211,1.86,"2 km SSW of Tres Pinos, CA",2023-08-17 18:58:16.770,2023-08-17 19:11:11.520,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,53,25.0,0.09,118.0,md,"M 1.9 - 2 km SSW of Tres Pinos, CA",-121.331497,36.773998,5.61
2,ak023aj145m6,2.50,"8 km WNW of Houston, Alaska",2023-08-17 18:55:42.061,2023-08-17 18:57:15.078,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,96,NaN,0.45,NaN,ml,"M 2.5 - 8 km WNW of Houston, Alaska",-149.974300,61.645400,25.60
3,ak023aj0vlpp,2.40,"36 km NW of Valdez, Alaska",2023-08-17 18:15:51.837,2023-08-17 18:17:34.650,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,89,NaN,0.93,NaN,ml,"M 2.4 - 36 km NW of Valdez, Alaska",-146.753800,61.390200,15.00
4,ak023aj0t643,1.40,"10 km NW of Trapper Creek, Alaska",2023-08-17 18:04:16.512,2023-08-17 18:06:54.312,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,30,NaN,0.54,NaN,ml,"M 1.4 - 10 km NW of Trapper Creek, Alaska",-150.346200,62.391300,63.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10586,av91110943,0.37,"71 km WSW of Nikolski, Alaska",2023-07-18 19:57:20.320,2023-07-21 02:26:55.280,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,2,5.0,0.23,179.0,ml,"M 0.4 - 71 km WSW of Nikolski, Alaska",-169.903833,52.809333,5.42
10587,av91110938,0.08,"Fox Islands, Aleutian Islands, Alaska",2023-07-18 19:56:47.700,2023-07-21 02:19:56.650,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0,4.0,0.13,333.0,ml,"M 0.1 - Fox Islands, Aleutian Islands, Alaska",-169.847000,52.787000,7.02
10588,us6000kvpn,4.60,"145 km SSE of ‘Ohonua, Tonga",2023-07-18 19:51:40.963,2023-08-09 16:18:14.040,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,326,22.0,0.54,90.0,mb,"M 4.6 - 145 km SSE of ‘Ohonua, Tonga",-174.377600,-22.544000,10.00
10589,hv73487487,1.82,"Island of Hawaii, Hawaii",2023-07-18 19:51:13.690,2023-07-18 19:54:14.840,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,51,24.0,0.10,126.0,md,"M 1.8 - Island of Hawaii, Hawaii",-155.469162,19.187334,30.23


In [36]:
# updated_df['tz']

In [37]:
updated_df.rename(columns={'properties.mag':'Magnitude','properties.place':'Location','properties.tz':'tz'})

# flatten_dict_df['geometry.coordinates'][0][0]


,id,Magnitude,Location,Timestamp,Updated_Time,properties.url,properties.detail,properties.sig,properties.nst,properties.rms,properties.gap,properties.magType,properties.title,longitude,latitude,depth_km
0,ak023aj1iz4s,1.90,Alaska Peninsula,2023-08-17 19:24:56.609,2023-08-17 19:26:33.554,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,56,NaN,0.34,NaN,ml,M 1.9 - Alaska Peninsula,-154.966200,58.305200,3.70
1,nc73925211,1.86,"2 km SSW of Tres Pinos, CA",2023-08-17 18:58:16.770,2023-08-17 19:11:11.520,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,53,25.0,0.09,118.0,md,"M 1.9 - 2 km SSW of Tres Pinos, CA",-121.331497,36.773998,5.61
2,ak023aj145m6,2.50,"8 km WNW of Houston, Alaska",2023-08-17 18:55:42.061,2023-08-17 18:57:15.078,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,96,NaN,0.45,NaN,ml,"M 2.5 - 8 km WNW of Houston, Alaska",-149.974300,61.645400,25.60
3,ak023aj0vlpp,2.40,"36 km NW of Valdez, Alaska",2023-08-17 18:15:51.837,2023-08-17 18:17:34.650,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,89,NaN,0.93,NaN,ml,"M 2.4 - 36 km NW of Valdez, Alaska",-146.753800,61.390200,15.00
4,ak023aj0t643,1.40,"10 km NW of Trapper Creek, Alaska",2023-08-17 18:04:16.512,2023-08-17 18:06:54.312,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,30,NaN,0.54,NaN,ml,"M 1.4 - 10 km NW of Trapper Creek, Alaska",-150.346200,62.391300,63.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10586,av91110943,0.37,"71 km WSW of Nikolski, Alaska",2023-07-18 19:57:20.320,2023-07-21 02:26:55.280,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,2,5.0,0.23,179.0,ml,"M 0.4 - 71 km WSW of Nikolski, Alaska",-169.903833,52.809333,5.42
10587,av91110938,0.08,"Fox Islands, Aleutian Islands, Alaska",2023-07-18 19:56:47.700,2023-07-21 02:19:56.650,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0,4.0,0.13,333.0,ml,"M 0.1 - Fox Islands, Aleutian Islands, Alaska",-169.847000,52.787000,7.02
10588,us6000kvpn,4.60,"145 km SSE of ‘Ohonua, Tonga",2023-07-18 19:51:40.963,2023-08-09 16:18:14.040,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,326,22.0,0.54,90.0,mb,"M 4.6 - 145 km SSE of ‘Ohonua, Tonga",-174.377600,-22.544000,10.00
10589,hv73487487,1.82,"Island of Hawaii, Hawaii",2023-07-18 19:51:13.690,2023-07-18 19:54:14.840,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,51,24.0,0.10,126.0,md,"M 1.8 - Island of Hawaii, Hawaii",-155.469162,19.187334,30.23


In [38]:
# Parse Strings to get locations: need Regex here is an example to view
updated_df[updated_df['properties.place'].str.contains('colombia',case=False)]

,id,properties.mag,properties.place,Timestamp,Updated_Time,properties.url,properties.detail,properties.sig,properties.nst,properties.rms,properties.gap,properties.magType,properties.title,longitude,latitude,depth_km
8,us7000kp2s,4.8,"12 km NW of Villavicencio, Colombia",2023-08-17 17:40:16.323,2023-08-17 19:19:20.040,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,354,81.0,1.17,70.0,mb,"M 4.8 - 12 km NW of Villavicencio, Colombia",-73.6879,4.2333,10.000
15,us7000kp2j,5.7,"6 km SW of Restrepo, Colombia",2023-08-17 17:17:18.031,2023-08-17 19:42:33.639,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,1017,79.0,0.98,77.0,mww,"M 5.7 - 6 km SW of Restrepo, Colombia",-73.6065,4.2142,7.942
18,us7000kp2i,6.3,"16 km N of Cumaral, Colombia",2023-08-17 17:04:49.641,2023-08-17 19:46:43.798,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,786,131.0,1.35,39.0,mww,"M 6.3 - 16 km N of Cumaral, Colombia",-73.5111,4.4177,10.000
54,us7000kp07,4.6,"22 km WNW of Valdivia, Colombia",2023-08-17 13:28:36.571,2023-08-17 18:30:34.936,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,328,49.0,0.82,105.0,mb,"M 4.6 - 22 km WNW of Valdivia, Colombia",-75.5938,7.3366,54.138
938,us6000kzwn,4.9,"15 km WNW of Valdivia, Colombia",2023-08-14 15:55:10.597,2023-08-17 17:43:57.809,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,371,158.0,0.80,83.0,mb,"M 4.9 - 15 km WNW of Valdivia, Colombia",-75.5221,7.3477,48.084
1278,us6000kzpp,4.3,"136 km WNW of Salahonda, Colombia",2023-08-13 12:58:42.786,2023-08-13 13:32:41.040,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,284,33.0,0.75,137.0,mb,"M 4.3 - 136 km WNW of Salahonda, Colombia",-79.6821,2.7223,10.000
1757,us6000kzf4,4.9,"19 km WNW of Valdivia, Colombia",2023-08-11 17:59:59.210,2023-08-12 15:34:01.197,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,370,93.0,0.72,134.0,mwr,"M 4.9 - 19 km WNW of Valdivia, Colombia",-75.5560,7.3617,46.685
1975,us6000kz6d,4.3,"13 km N of Jordán, Colombia",2023-08-11 01:27:15.106,2023-08-11 02:26:26.040,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,284,32.0,0.87,129.0,mb,"M 4.3 - 13 km N of Jordán, Colombia",-73.0839,6.8585,145.692
3328,us6000ky9h,4.3,"11 km SE of Piedecuesta, Colombia",2023-08-06 11:09:28.786,2023-08-06 11:23:54.040,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,284,47.0,0.75,112.0,mb,"M 4.3 - 11 km SE of Piedecuesta, Colombia",-72.9849,6.9055,143.798
4013,us6000kxr4,4.2,"11 km SW of Piedecuesta, Colombia",2023-08-04 05:47:15.686,2023-08-16 12:21:34.040,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,271,46.0,0.65,126.0,mb,"M 4.2 - 11 km SW of Piedecuesta, Colombia",-73.1242,6.9172,144.024


In [39]:
# Remove digit, km, direction first

g=[]
for i in updated_df['properties.place'].to_list():
    # remove all text up to word "of" if it is not there then ignore and append
    g.append(re.sub('^.*of',"", i).strip())

# Update this column with fixed location formatting (still you can adjust further if needed)

updated_df['properties.place']=g
updated_df

,id,properties.mag,properties.place,Timestamp,Updated_Time,properties.url,properties.detail,properties.sig,properties.nst,properties.rms,properties.gap,properties.magType,properties.title,longitude,latitude,depth_km
0,ak023aj1iz4s,1.90,Alaska Peninsula,2023-08-17 19:24:56.609,2023-08-17 19:26:33.554,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,56,NaN,0.34,NaN,ml,M 1.9 - Alaska Peninsula,-154.966200,58.305200,3.70
1,nc73925211,1.86,"Tres Pinos, CA",2023-08-17 18:58:16.770,2023-08-17 19:11:11.520,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,53,25.0,0.09,118.0,md,"M 1.9 - 2 km SSW of Tres Pinos, CA",-121.331497,36.773998,5.61
2,ak023aj145m6,2.50,"Houston, Alaska",2023-08-17 18:55:42.061,2023-08-17 18:57:15.078,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,96,NaN,0.45,NaN,ml,"M 2.5 - 8 km WNW of Houston, Alaska",-149.974300,61.645400,25.60
3,ak023aj0vlpp,2.40,"Valdez, Alaska",2023-08-17 18:15:51.837,2023-08-17 18:17:34.650,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,89,NaN,0.93,NaN,ml,"M 2.4 - 36 km NW of Valdez, Alaska",-146.753800,61.390200,15.00
4,ak023aj0t643,1.40,"Trapper Creek, Alaska",2023-08-17 18:04:16.512,2023-08-17 18:06:54.312,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,30,NaN,0.54,NaN,ml,"M 1.4 - 10 km NW of Trapper Creek, Alaska",-150.346200,62.391300,63.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10586,av91110943,0.37,"Nikolski, Alaska",2023-07-18 19:57:20.320,2023-07-21 02:26:55.280,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,2,5.0,0.23,179.0,ml,"M 0.4 - 71 km WSW of Nikolski, Alaska",-169.903833,52.809333,5.42
10587,av91110938,0.08,"Fox Islands, Aleutian Islands, Alaska",2023-07-18 19:56:47.700,2023-07-21 02:19:56.650,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,0,4.0,0.13,333.0,ml,"M 0.1 - Fox Islands, Aleutian Islands, Alaska",-169.847000,52.787000,7.02
10588,us6000kvpn,4.60,"‘Ohonua, Tonga",2023-07-18 19:51:40.963,2023-08-09 16:18:14.040,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,326,22.0,0.54,90.0,mb,"M 4.6 - 145 km SSE of ‘Ohonua, Tonga",-174.377600,-22.544000,10.00
10589,hv73487487,1.82,"Hawaii, Hawaii",2023-07-18 19:51:13.690,2023-07-18 19:54:14.840,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,51,24.0,0.10,126.0,md,"M 1.8 - Island of Hawaii, Hawaii",-155.469162,19.187334,30.23


# `Threading Example:`

In [40]:
def download_file(url_list):
    sd_=[]
    for i in url_list:

        se_inverterdata_ = 'https://monitoringapi.solaredge.com/equipment/'+ i[0] + '/'+ i[1] +'/data?startTime='+ i[3] +'%2000:00:00&endTime=' + i[4] + '%2023:59:00&api_key=' + i[2]
#             if se_inverterdata_:
        if requests.get(se_inverterdata_ ).json():
#                 inverterdata_p = requests.get(se_inverterdata_).json()['data']['telemetries']
            sd_.append([i[0],requests.get(se_inverterdata_).json()['data']['telemetries']])
        else:
            sd_.append([i[0],'Nope'])
    return sd_


start = time()

processes = []
oo=[]
with ThreadPoolExecutor(max_workers=10) as executor:
    for url_list in store_urls[:20]:
        for j in df_dates_xx[:20]:
#         se_inverterdata_ = 'https://monitoringapi.solaredge.com/equipment/'+ i[0] + '/'+ i[1] +'/data?startTime='+ i[3] +'%2000:00:00&endTime=' + i[4] + '%2023:59:00&api_key=' + i[2]

            processes.append((executor.submit(download_file, url_list)))
            oo.append([i[1],processes])
p_=[]
for task in as_completed(processes):
    p_.append(task.result())


print(f'Time taken: {time() - start}')

TypeError: 'module' object is not callable

In [18]:
# Dynamic Program Parsing this example:


In [20]:
# pd.to_datetime(flatten_dict_df['properties.updated'],unit='ms').dt.tz_localize('UTC')

# Like, Share & <font color=red>SUB</font>scribe

In [ ]:
# url = requests.get(f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_time}&minmagnitude={min_magnitude}&latitude={latitude}&longitude={longitude}&maxradiuskm={max_radius_km}')
# 

# `Citations & Help:`

# ◔̯◔

https://softhints.com/how-to-send-multiple-concurrent-requests-in-python/

https://medium.com/mlearning-ai/analyzing-and-visualizing-earthquake-data-received-with-usgs-api-in-python-environment-8c7e7b6bed02

http://rits.github-pages.ucl.ac.uk/doctoral-programming-intro/02-novice/060EarthquakesSolution.html

https://cartovista.com/real-time-data-cartovista-api-python/

https://foongminwong.medium.com/converting-unix-timestamp-to-readable-datetime-format-ada8fff362fe

https://santhoshm.hashnode.dev/usgs-api-to-map-earthquake

https://www.codespeedy.com/analyze-and-visualize-earthquake-data-in-python-with-matplotlib/

https://risk-engineering.org/notebook/data-analysis-earthquakes.html

https://stackoverflow.com/questions/35491274/split-a-pandas-column-of-lists-into-multiple-columns

https://stackoverflow.com/questions/25045373/use-regex-re-sub-to-remove-everything-before-and-including-a-specified-word

https://medium.com/mlearning-ai/analyzing-and-visualizing-earthquake-data-received-with-usgs-api-in-python-environment-8c7e7b6bed02